# Week 5 - NLP and Deep Learning

---

# Lecture 9: Sequence Prediction with HMMs

In this exercise you will implement the Viterbi algorithm for decoding in sequence tagging. More concretely, we are going to build a POS tagger for English web data.

## 1. Emissions and transition probabilities

In this part of the exercise you are going to prepare the emission and transition probabilities to use in the viterbi algorithm. We are going to focus on the task of Parts-Of-Speech (POS) tagging. You can use the following datareader for the following assignments:

In [1]:
def read_conll_file(path):
    """
    read in conll file
    
    :param path: path to read from
    :returns: list with sequences of words and labels for each sentence
    """
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue # skip comments
            tok = line.split('\t')

            current_words.append(tok[0])
            current_tags.append(tok[1])
        else:
            if current_words:  # skip empty lines
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    # check for last one
    if current_tags != []:
        data.append((current_words, current_tags))
    return data

Because we are going to use the POS labels as indices in our Viterbi matrix, we need to know all labels beforehand, and they need to have a static order. We also need a special beginning and end label:

In [2]:
train_data = read_conll_file('pos-data/en_ewt-train.conll')
dev_data = read_conll_file('pos-data/en_ewt-dev.conll')

SMOOTH = 0.1
BEG = '<S>'
END = '</S>'
UNK = '<UNK>'

label_set = set([pos_label for sentence in train_data for pos_label in sentence[1]])
label_set.add(BEG)
label_set.add(END)
# put labels in a list, so that they are guaranteed to have the same order
label_list = list(sorted(label_set))

print('Length train data: ' + str(len(train_data)))
print('Length dev data: ' + str(len(dev_data)))

print('All labels:')
print(label_list)

# the data is a list of pairs, containing 1: a list of words 2: a list of POS labels
print('Random datapoint:')
print(train_data[70][0])
print(train_data[70][1])



Length train data: 12543
Length dev data: 2000
All labels:
['</S>', '<S>', 'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
Random datapoint:
['It', 'is', 'a', 'time', 'to', 'learn', 'what', 'happened', 'and', 'how', 'it', 'may', 'affect', 'the', 'future', '.']
['PRON', 'AUX', 'DET', 'NOUN', 'PART', 'VERB', 'PRON', 'VERB', 'CCONJ', 'ADV', 'PRON', 'AUX', 'VERB', 'DET', 'NOUN', 'PUNCT']


* a) calculate the transition probabilities based on the training data, use a special label for the beginning of a sentence (`<S>`) and the end of a sentence (`</S>`). use laplace smoothing with a value of 0.1 to avoid probabilities of 0.0. In this case the UNK tag has a frequency of 0.1 to all all other tags (in most situations an UNK label is not to be expected to occur, but some POS tags can be rare).

**Hint**: The transition probability $P(t_i|t_{i-1})$ is the probability that given a tag, $t_{i-1}$, that it will be followed by a tag $t_i$. 
$$P(t_i|t_{i-1}) = {C(t_{i-1},t_{i}) \over C(t_{i-1})}$$
With smoothing:
$$P(t_i|t_{i-1}) = {C(t_{i-1},t_{i}) + \gamma \over C(t_{i-1}) + (|t|) * \gamma} $$

Where $(|t|-1) * \gamma$ is used because we want add probability mass to all labels in the numerator (we have to match this mass in the denominator).

**Hint2**: Every sentence in the data looks like `['DET', 'NOUN', 'VERB']` without any `<S>` or `</S>` tags. So the beginning and end of every data sample needs to be handled differently when counting occurences of transitions, alternatively you can add these tokens to each data sample so they look like `['<S>, 'DET', 'NOUN', 'VERB', </S>]`

In [ ]:
# Make a dictionary of dictionaries, so that we can easily query for a certain probability.
# We add the smoothing value as a starting point, as it has to be added to each count.
# Note that a list of lists with POS indices is more efficient (but a bit more cumbersome to implement).
transition_counts = {label: {label: SMOOTH for label in label_list} for label in label_list}
# The count that a NOUN follows an ADJ (empty now)
print(transition_counts['ADJ']['NOUN'])

# First obtain the raw counts
# TODO

print(transition_counts['ADJ']['NOUN']) # should be 6803.1

# Now fill the transition matrix, note that the outgoing probability of each label should sum to 1.0
transition_probs = {label: {label: 0.0 for label in label_list} for label in label_list}
# TODO

        
print(transition_probs['ADJ']['NOUN']) # 0.5191660498019673
print(sum(transition_probs['ADJ'].values())) # 1.0

* b) calculate the emission probabilities for all words in the training data. Make sure that every POS tag can be assigned to an `<UNK>` token, use laplace smoothing with a value of 0.1 to avoid probabilities of 0.0.

**Hint**: The emission probability $P(w_i|t_{i})$ is the probability that given a tag, $t_i$, that it will be associated with a given word $w_i$. The formula below shows counts $C$ needed to calculate the probability.

$$P(w_i|t_{i}) = {C(t_{i},w_{i}) \over C(t_{i})}$$

With Smoothing:

$$P(w_i|t_{i}) = {C(t_{i},w_{i}) + \gamma \over C(t_{i}) + (|t|) * \gamma}$$

In [ ]:
word_set = {UNK}
for sent in train_data:
    for word in sent[0]:
        word_set.add(word)
word_list = list(sorted(word_set))

# Fill emission counts, start with smoothing value again
emission_counts = {label: {word: SMOOTH for word in word_list} for label in label_list}
# TODO


print(emission_counts['ADJ']['European']) # 6.1
print(emission_counts['ADJ']['Europe']) # 0.1

# Convert to probabilities
emission_probs = {label: {word: 0.0 for word in word_list} for label in label_list}
# TODO
      

print(emission_probs['ADJ']['European']) # 0.00040346316910398104
print(emission_probs['ADJ']['Europe']) # 6.614150313180017e-06

You can check whether your solutions are correct by estimating the probabilities on the data and check whether the probabilities match:

In [ ]:
print(transition_probs['ADJ']['NOUN']) # 0.5171926196793345
print(transition_probs['NOUN']['ADJ']) # 0.01123434129302644
print(transition_probs[BEG]['ADJ']) # 0.04082136964025221
print(transition_probs['ADJ'][END]) # 0.003808756338424345
print(emission_probs['NOUN']['calling'])   # 2.9909103515414666e-05
print(emission_probs['VERB']['calling'])  # 0.0005740785225418476
print(emission_probs['VERB'][UNK])  # 4.071478883275515e-06

## 2. Viterbi algorithm

In the image below we see an example of the calculation of the first 2 positions in a Viterbi decoding:
<img width=500px src="pics/viterbi.jpg">

* a) Implement Viterbi decoding, use the transition and emission probabilities previously estimated (note that we also provide pre-calculated probabilities in `probs_en.pickle`). You can use the example code shown below as a starting point if you like.

**Hint**: The implementation can become simpler if you think about the problem as a 2d matrix that needs to be filled (each position in the list is a node in the viterbi decoding, $v_1(7)$, $v_1(6)$, ...). You can first initialize the matrix with 0.0's, and then fill it from left to right.

In [ ]:
# You can also load the pre-calculate probabilities:
# import pickle
# transition_probs, emission_probs = pickle.load(open('probs_en.pickle', 'rb'))
import numpy as np
def viterbi(sentence):
    """
    sentence: list of strings
    """
    columns = len(sentence)
    # You don't need the special tokens in the viterbi decoding so we remove them
    labels_list_exc_special = label_list.copy()
    labels_list_exc_special.remove(BEG)
    labels_list_exc_special.remove(END)

    rows = len(labels_list_exc_special)
    
    # Create the full matrix for scores as well as the backtracking.
    # e.g. scores[0][3] should get the probability of the best path of 
    # the first label and the 4th word in the sentence
    # Backtrack contains the index of the best tag for the previous word
    # e.g. backtrack[0][3] should get the index of the best tag for the 3rd word 
    # when backtracking from the first label and 4th word
    scores = np.array([[0.0 for _ in range(columns)] for _ in range(rows)])
    backtrack = np.array([[0 for _ in range(columns)] for _ in range(rows)])
    
    # Handle the first token separately, as it only has 2 probabilities (emission, transition)
    word_position = 0
    for pos_tag_idx, pos_tag in enumerate(labels_list_exc_special):
        # The probability of the first word given the POS tag:
        word = sentence[word_position]
        if word not in emission_probs[pos_tag]:
            word = UNK
        em_prob = emission_probs[pos_tag][word] 
        
        # The probability of the POS tag given that the previous "tag" is <S>
        transition_prob = transition_probs[BEG][pos_tag]
        
        # Save the total probability:
        scores[pos_tag_idx][word_position] = em_prob * transition_prob
        
        # Backtracking for the first token is uneccessary so we ignore it
    
    # Now handle the rest of the sequence
    for word_position in range(1, columns):
        for pos_tag_idx, pos_tag in enumerate(labels_list_exc_special):
            word = sentence[word_position]
            if word not in emission_probs[pos_tag]:
                word = UNK

            # Get emission probability
            # TODO

            # For each possible history path (i.e. label): get the total score
            # We need to get the transition probability and the history probability
            # Hint: the history probability is the score of the previous word position in scores matrix
            # TODO
            candidate_scores = [0.0] * len(labels_list_exc_special)
            
                
            # Now extract the best score from candidate_scores and its previous path and save these
            # Hint: backtrack should contain the index of the best tag:
            # backtrack[tag_idx][word_position] = previous_best_tag_idx
            # TODO
            

    # Extract the best score from the last labels to the special end label
    # Hint: here you only have history and transition (no emission)
    # TODO
    
    
    # Extract the path from the best last label using the backtrack matrix
    # Hint: the path contains the index of the best tag for each word
    # TODO
    current_path = [best_prev_idx]
    
    
    # Reverse the path and convert the indexes to labels
    # TODO
    
    
    return 

viterbi(['this', 'is', 'a', 'very', 'good', 'chocolate', '.'])


* b) Ensure that the best path is saved during the decoding, so that you can extract the labels. What is the accuracy (code for this below) of your implementation of the Viterbi algorithm on the development data (`pos-data/en_ewt-dev.conll`)?

**Hint**: If implemented correctly, it should score at least an accuracy of 50%. If you score lower, we suggest you try printing the probabilities at each step (word) for the first sentence of the development data.

In [ ]:
correct = 0
total = 0

# Loop through the development data and compute predictions
for words, true_tags in dev_data:
    predicted_tags = viterbi(words)  # Get the POS tags from your Viterbi algorithm
    # Compare predicted tags with true tags
    correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
    total += len(true_tags)

# Calculate the accuracy
accuracy = correct / total if total > 0 else 0
print(f"Viterbi Accuracy on Development Data: {accuracy * 100:.2f}%")

* c) **Bonus**: try to improve your predictions by inspecting common errors, tuning some of the decisions (e.g. smoothing, weighing the three probabilities) you made, or improving the handling of unknown tokens.


# Lecture 10: BERT

## 3. Subword tokenization

BERT models are trained to predict tokens that were masked with a special `[mask]` token. In this assignment you will inspect what it has learned, and whether it has certain preferences (i.e. probing). 

a) Load the multilingual Bert tokenizer:

In [ ]:
# ! pip install transformers

     ---------------------------------------- 0.0/44.0 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.0 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 44.0/44.0 kB 718.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
    --------------------------------------- 0.2/10.0 MB 4.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/10.0 MB 5.3 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/10.0 MB 7.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.7/10.0 MB 9.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.3/10.0 MB 9.8 MB/s eta 0:00:01
   ----------- ---------------------------- 2.9/10.0 MB 10.3 MB/s eta 0:00:01
   -------------- ------------------------- 3.5/10.0 MB 10.7 MB/s eta 0:00:01
   --------------- ------------------------ 4.0/10.0 MB 10.2 MB/s eta 0:00:01
   ------------------ --------------------- 4.6/10.0 MB 10.5 MB/s eta 0:00:01
 

In [3]:
from transformers import AutoTokenizer

tokzr = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', use_fast=False)

c:\Users\luke0\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Multilingual BERT was trained on the 100 most frequent languages of Wikipedia. They used smoothing, to correct inbalances in the data. However, their smoothing is relatively conservative, so high-resource languages have a higher impact on the model, and it is unclear how they sampled for training the tokenizer. Compare the tokenizations for two different language types you know; preferably one higher-resource and one lower-resource. If you only know 1 language, or only high-resource languages, try to use a different variety of the language (for example for English, use social media abbreviations or typos, e.g.: c u tmrw). Can you observe any differences in the results? does it match your intuition of separating mostly meaning-carrying subwords?

You can use Figure 1 of https://arxiv.org/pdf/1911.02116.pdf or https://en.wikipedia.org/wiki/List_of_Wikipedias to see how large languages are on Wikipedia.

My test was between English and Te Reo (Maori), looking at the same sentence from the Treaty of Waitangi. The english is the new direct translation, not from the original English document.

In [4]:
tokzr.tokenize('So the Queen desires to establish a government so that no evil will come to Maori and European living in a state of lawlessness.')

['So',
 'the',
 'Queen',
 'desire',
 '##s',
 'to',
 'establish',
 'a',
 'government',
 'so',
 'that',
 'no',
 'evil',
 'will',
 'come',
 'to',
 'Mao',
 '##ri',
 'and',
 'European',
 'living',
 'in',
 'a',
 'state',
 'of',
 'law',
 '##less',
 '##ness',
 '.']

In [5]:
tokzr.tokenize('Na ko te Kuini e hiahia ana kia wakaritea te Kawanatanga kia kaua ai nga kino e puta mai ki te tangata maori ki te Pakeha e noho ture kore ana.')

['Na',
 'ko',
 'te',
 'Kui',
 '##ni',
 'e',
 'hi',
 '##ahi',
 '##a',
 'ana',
 'ki',
 '##a',
 'wa',
 '##kari',
 '##tea',
 'te',
 'Ka',
 '##wana',
 '##tang',
 '##a',
 'ki',
 '##a',
 'ka',
 '##ua',
 'ai',
 'nga',
 'kino',
 'e',
 'puta',
 'mai',
 'ki',
 'te',
 'tan',
 '##gata',
 'mao',
 '##ri',
 'ki',
 'te',
 'Pak',
 '##eh',
 '##a',
 'e',
 'noho',
 'tur',
 '##e',
 'kor',
 '##e',
 'ana',
 '.']

Interestingly, in English, the only word tokenised 'incorrectly', other than compound words is the word Maori.  
The Maori sentence has a lot more original word segmentation in the tokenisation.

This particular Maori sentence is also more likely than most to be understood, since it's from one of the most well known Maori language documents.

b) Test whether the `bert-base-cased` model can solve the analogy task that we discussed in the word2vec lecture ([slides](https://github.itu.dk/robv/intro-nlp2023/blob/main/slides/07-vector_semantics.pdf), [assignment](https://github.itu.dk/robv/intro-nlp2023/blob/main/assignments/week4/week4.ipynb)), we can do this by masking the target word we are looking for, and let the model predict which words fit best. We can then use a prompt to discover what the language model would guess. For example, we can use the prompt "man is to king as woman is to [MASK]". Try at least two syntactic analogies, and two semantic analogies.
You can use the following code:

(Note that you need 4gb of RAM for this assignment, otherwise you can use the HPC.)

In [6]:
from transformers import AutoModelForMaskedLM,AutoTokenizer
import torch

def getTopN(inputSent, model, tokzr, topn=1):
    maskId = tokzr.convert_tokens_to_ids(tokzr.mask_token)
    tokenIds = tokzr(inputSent).input_ids
    if maskId not in tokenIds:
        return 'please include ' + tokzr.mask_token + ' in your input'
    maskIndex = tokenIds.index(maskId)
    logits = model(torch.tensor([tokenIds])).logits
    return tokzr.convert_ids_to_tokens(torch.topk(logits, topn, dim=2).indices[0][maskIndex])

model = AutoModelForMaskedLM.from_pretrained('bert-base-cased')
tokzr = AutoTokenizer.from_pretrained('bert-base-cased')

getTopN('This is a [MASK] test.', model, tokzr, 5)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['positive', 'simple', 'negative', 'diagnostic', 'successful']

In [7]:
getTopN('man is to doctor as woman is to [MASK].', model, tokzr, 5)

['man', 'cook', 'nurse', 'judge', 'doctor']

In [8]:
getTopN('man is to king as woman is to [MASK].', model, tokzr, 5)

['king', 'queen', 'man', 'slave', 'rule']

In [9]:

#syntactic
print(getTopN('The [MASK] is here.', model, tokzr, 5))
print(getTopN('Here is the [MASK].', model, tokzr, 5))


print(getTopN('In the future, he will [MASK].', model, tokzr, 5))
print(getTopN('In the future he will [MASK].', model, tokzr, 5))


#semantic
print(getTopN('The tall man has the job title of [MASK].', model, tokzr, 5))
print(getTopN('The tall woman has the job title of [MASK].', model, tokzr, 5))

print(getTopN('Every Maori person [MASK].', model, tokzr, 5))
print(getTopN('Every Pakeha person [MASK].', model, tokzr, 5))

['answer', 'key', 'truth', 'body', 'girl']
['example', 'list', 'article', 'information', 'link']
['die', 'return', 'live', 'be', 'recover']
['die', 'return', 'be', 'live', 'recover']
['manager', 'CEO', 'supervisor', 'Sergeant', 'secretary']
['secretary', 'housekeeper', 'nurse', 'manager', 'waitress']
['died', 'did', 'knew', 'laughed', 'knows']
['died', 'dies', 'agrees', 'agreed', 'won']


c) Test how robust the language model is, does it have an effect on the results of the word predictions if you include punctuations at the end of the sentence?, what about starting with a capital? and do typos have a large impact?

d) Think of some prompts that test whether the model has any gender biases, you can test this for example by using common gendered names or pronouns, swapping them and then check whether the predicted word changed.

# 4. Finetune a BERT model

We have provided code for training a BERT based classifier, which can be found in `week5/bert/bert-topic.py`. The implementation uses huggingface's transformers library (https://github.com/huggingface/transformers), and simply adds a linear layer to convert the output of the CLS token from the last layer of the masked language model to a label. 

You can install the transformers library with the command:
`pip3 install transformers`

a) Inspect the code; what should the shape of the output_scores be at the end of the forward pass?, What does this output represent?

b) Train the model on the HPC without a GPU, how long does it take? You can use the `scavenge` queue, and note that if you do not specify a GPU it will not use one. You can use a job script like:

```
#!/bin/bash

#SBATCH --job-name=bert-small
#SBATCH --output=bert-small.out
#SBATCH --cpus-per-task=2
#SBATCH --time=10:00:00
#SBATCH --mem=12G
#SBATCH --mail-type=BEGIN,END,FAIL
#SBATCH --partition=scavenge

python3 # TODO
```
More information about the ITU HPC can be found on: https://hpc.itu.dk/ (only available on ITU network/VPN).

c) Now change the number of maximum training sentences (MAX_TRAIN_SENTS) to 500 and the batch size (BATCH_SIZE) to 32. It will now take very long to train without a GPU. Train the model on the HPC, make sure you reserve a GPU to speed up the training, this can be done with `#SBATCH --gres=gpu`. Note that the code detects automatically whether a GPU is available.